# Previous experiments

In [1]:
from transformers import BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel, BertTokenizer, TrainingArguments, Trainer, Seq2SeqTrainer, Seq2SeqTrainingArguments 
from datasets import Dataset
import pandas as pd
import numpy as np
import torch

import os
import math 

from hana_ml import dataframe
from dotenv import load_dotenv

from rouge import Rouge
rouge = Rouge()

# MEANSUM model for cluster summary

In [333]:
load_dotenv()
conn = dataframe.ConnectionContext(address='localhost', port=30015, user=os.getenv('hana_user'), password=os.getenv('hana_password'))

In [334]:
clusters = conn.table('SAP_NEWS_CENTER_TOPICCLUSTERS', schema=os.getenv('hana_nc_schema'))
df_108 = clusters.filter("CLUSTER_LABEL = 108").select("START_DATE", "ARTICLE_ID").collect()

ids = ','.join(["'"+id+"'" for id in df_108['ARTICLE_ID'].tolist()])
df_108_articles = conn.table('SAP_NEWS_CENTER_ARTICLES', schema=os.getenv('hana_nc_schema')).filter(f"ID in ({ids})").collect()

cluster_content = df_108_articles.iloc[[1,2,3,4,5,8,12,13]]['CONTENT'].tolist()

In [2]:
autoencoder = EncoderDecoderModel.from_pretrained('../saved-models/bert2bert/content-reconstruction/v3')
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

encoder = autoencoder.get_encoder()
decoder = autoencoder.get_decoder()

In [3]:
cont = tokenizer(['Mein Name ist Marco und ich spiele gerne Fussball'], return_tensors="pt")
gen = autoencoder.generate(cont.input_ids)
tokenizer.decode(gen[0])

'[CLS] Mein Name ist Marco und ich spiele gerne Fussballballball [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] mein Name ist Marco und ich spiele gerne Fussball ist Marco und ich spiele gerne Fussballball [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] mein Name ist [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] mein Name ist Marco und ich spiele gerne spiele gerne Fussballe gerne Fussballball [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] 

In [326]:
tokenized_content = tokenizer(cluster_content, padding="max_length", truncation=True, max_length=512, add_special_tokens=True, return_tensors="pt")

In [327]:
encoded_vectors = encoder(tokenized_content.input_ids, attention_mask=tokenized_content.attention_mask).last_hidden_state
mean = torch.mean(encoded_vectors, dim=0, keepdim=True)

decoder_input_ids = torch.tensor([[autoencoder.config.encoder.bos_token_id]])

In [328]:
i = 0
max_i = 512

while i < max_i:
    i = i+1
    outputs = decoder(decoder_input_ids, encoder_hidden_states=mean)
    next_decoder_input_ids = torch.argmax(outputs.logits[:, -1:], axis=-1)
    decoder_input_ids = torch.cat([decoder_input_ids, next_decoder_input_ids], axis=-1)
    if next_decoder_input_ids[0][0] == decoder.config.pad_token_id:
        print("End of Summary!")
        break
    
summary = tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True)
summary

'Der Vorsitzende der Ständigen Impfkommission in dere, der der, der der Ständigen Der Chef der Vorsitzende der Vorsitzende, der der Ständigen, der für die die Der der Vorsitzende der Vorsitzende dere der der und der von Der bei der Vorsitzende der die in den auf den Impfstoff für der Empfehlung der Ständigen, und die die die in den den Menschen nach der Ständigen, dass der Impfung von der Impfung für die die in der der und sind, der der Ständige die die und die Empfehlung der Ständigen Impfung in der in der der und die für der Ständigen Impfung, die der die die Impfung von der EU - Stiko -, der der der Ständigen und die in der in der der, die die nach der die nach der Ständigen die nach der und die für die Corona - die die wegen der und die für der, wie die drei Millionen Millionen Menschen als eine von der in der die, wie die'

# Adapted MEANSUM Theory for unsupervised single document summary

## Experiment 1
Try to split the text tokens into target summary size and reconstruct the texts for this size
Model is trained for 512 token length, but we will try if it is capable of reconstructing e.g. 128 token length text

In [336]:
# Take any text for experiment
text = cluster_content[3]
text

'STIKO-Mitglied Zepp hält eine begrenzte Empfehlung der Kinder-Impfungen für plausibel.\n(Foto: picture alliance / dpa)\nDie Bundesregierung will in der kommenden Woche, auch Kinder ab zwölf Jahre in die Impfkampagne einbeziehen. Die Zulassung eines Vakzins liegt bereits vor, aber noch fehlt die Empfehlung der STIKO. Deren Mitglied Zepp gibt im Interview mit ntv einen Einblick in die Überlegungen der Kommission.\nntv: Es wird wieder viel über Schüler diskutiert. Viele gehen seit gestern in ganzer Klassenstärke in die Schule. Ist das eine richtige Entscheidung?\nFred Zepp: Ja. Wir haben nach den Ferien empfohlen, den Schulbetrieb wieder zunehmen. Angesichts der deutlich rückläufigen Infektionslage plus der Rahmenbedingungen plus der Erfahrungen, die wir gemacht haben, dass es in Schulen besonders gut funktioniert mit den Hygiene-Regeln, ist es aus unserer Sicht zulässig. Vor allen Dingen ist es wichtig für Kinder und Jugendliche, endlich wieder in einen normalen Schulbetrieb zurückgefüh

In [337]:
tokenized_content = tokenizer(text, add_special_tokens=False, return_tensors="pt")
tokenized_content.input_ids.size()

Token indices sequence length is longer than the specified maximum sequence length for this model (1234 > 512). Running this sequence through the model will result in indexing errors


torch.Size([1, 1234])

In [340]:
split_size = int(math.ceil(1234/9))
splitted_input_ids = torch.split(tokenized_content.input_ids, split_size, 1)
splitted_attention_mask = torch.split(tokenized_content.attention_mask, split_size, 1)

input_data = []

splitted_input_ids = [torch.cat([torch.tensor([[autoencoder.config.encoder.bos_token_id]]), t], dim=1) for t in splitted_input_ids]
splitted_attention_mask = [torch.cat([torch.tensor([[1]]), t], dim=1) for t in splitted_attention_mask]

In [342]:
input_ids = torch.stack(list(splitted_input_ids[:8]), 1).squeeze()
attention_mask = torch.stack(list(splitted_attention_mask[:8]), 1).squeeze()

In [343]:
generated = autoencoder.generate(input_ids, attention_mask=attention_mask, max_length=split_size, decoder_start_token_id=3)

In [344]:
reconstructed_text = ' '.join(tokenizer.batch_decode(generated, skip_special_tokens=True))

In [345]:
reconstructed_text

'STIKO - Mitglied Zepp hält eine begrenzte Empfehlung der Kinder - Impfungen für plausibel. ( Foto : picture alliance / dpa ) Die Bundesregierung will in der kommenden Woche, auch Kinder ab zwölf Jahre in die Impfkampagne einbeziehen. Die Zulassung eines Vakzins liegt bereits vor, aber noch fehlt die Empfehlung der STIKO. Deren Mitglied Zepp gibt im Interview mit ntv einen Einblick in die Überlegungen der Kommission. ntv : Es wird wieder viel über Schüler diskutiert. Viele gehen seit gestern in ganzer Klassenstärke in die Schule. Ist das eine richtige Entscheidung? Fred Zepp : Ja. Wir haben nach den Ferien empfohlen den Schulbetrieb wieder zunehmen. Angesichts der deutlich rückläufigen Infektionslage plus der Rahmenbedingungen plus der Erfahrungen, die wir gemacht haben, dass es in Schulen besonders gut funktioniert mit den Hygiene - Regeln, ist es aus unserer Sicht zulässig. Vor allen Dingen ist es wichtig für Kinder und Jugendliche, endlich wieder in einen normalen Schulbetrieb zurüc

In [191]:
print(f'Length of original text: {len(text)}')
print(f'Length of reconstructed text: {len(reconstructed_text)}')

Length of original text: 2661
Length of reconstructed text: 2681


In [346]:
rouge.get_scores(text, reconstructed_text)

[{'rouge-1': {'r': 0.9545454545454546,
   'p': 0.8786610878661087,
   'f': 0.9150326747471297},
  'rouge-2': {'r': 0.9271781534460338,
   'p': 0.8642424242424243,
   'f': 0.8946047628857197},
  'rouge-l': {'r': 0.9545454545454546,
   'p': 0.8786610878661087,
   'f': 0.9150326747471297}}]

## Experiment 2
Try mean over encodings of the splitted input and then decode the mean vector

In [388]:
encoded_vectors = encoder(input_ids, attention_mask=attention_mask).last_hidden_state
mean = torch.mean(encoded_vectors, dim=0, keepdim=True)

decoder_input_ids = torch.tensor([[autoencoder.config.encoder.bos_token_id]])

In [331]:
encoder(input_ids, attention_mask=attention_mask).keys()

odict_keys(['last_hidden_state'])

In [387]:
test_out = decoder(torch.tensor([[3,4]]), output_hidden_states=True, encoder_hidden_states=mean[:,0,:].reshape(1,1,768))
conc = torch.cat([mean[:,0,:].reshape(1,1,768), test_out.hidden_states[-1][:,-1,:].reshape(1,1,768)], 1)
conc.size()

torch.Size([1, 2, 768])

In [386]:
test_out.hidden_states[-1][:,-1,:].reshape(1,1,768).size()

torch.Size([1, 1, 768])

In [389]:
i = 0
max_i = split_size

hs = mean[:,0,:].reshape(1,1,768)

while i < max_i:
    i = i+1
    outputs = decoder(decoder_input_ids, output_hidden_states=True, encoder_hidden_states=hs)
    hs = torch.cat([hs, outputs.hidden_states[-1][:,-1,:].reshape(1,1,768)], 1)
    next_token_logits = outputs.logits[:,-1:]
    next_tokens_scores = decoder._get_logits_processor(
        repetition_penalty=decoder.config.repetition_penalty,
        no_repeat_ngram_size=decoder.config.no_repeat_ngram_size,
        encoder_no_repeat_ngram_size=decoder.config.encoder_no_repeat_ngram_size,
        encoder_input_ids=None,
        bad_words_ids=None,
        min_length=decoder.config.min_length,
        max_length=decoder.config.max_length,
        eos_token_id=decoder.config.eos_token_id,
        forced_bos_token_id=decoder.config.forced_bos_token_id,
        forced_eos_token_id=decoder.config.forced_eos_token_id,
        prefix_allowed_tokens_fn=None,
        num_beams=decoder.config.num_beams,
        num_beam_groups=decoder.config.num_beam_groups,
        diversity_penalty=decoder.config.diversity_penalty,
        remove_invalid_values=decoder.config.remove_invalid_values,
    )(decoder_input_ids, next_token_logits)
    next_decoder_input_ids = torch.argmax(next_tokens_scores, axis=-1)
    decoder_input_ids = torch.cat([decoder_input_ids, next_decoder_input_ids], axis=-1)
    if next_decoder_input_ids[0][0] == decoder.config.pad_token_id:
        print("End of Summary!")
        break
    


In [391]:
summary = tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True)
summary

'[CLS] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]'

In [145]:
torch.cat([torch.tensor([[autoencoder.config.encoder.bos_token_id]]), splitted_input_ids[1]], dim=1).size()

torch.Size([1, 115])

In [231]:
outputs = decoder(torch.tensor([[3]]), encoder_hidden_states=torch.mean(encoded_vectors[0:2,:,:], dim=0, keepdim=True))

In [232]:
next_token_logits = outputs.logits[:,-1,:]

In [233]:
torch.argmax(next_token_logits, dim=-1)

tensor([114])

In [206]:
next_tokens_scores = decoder._get_logits_processor(
    repetition_penalty=decoder.config.repetition_penalty,
    no_repeat_ngram_size=decoder.config.no_repeat_ngram_size,
    encoder_no_repeat_ngram_size=decoder.config.encoder_no_repeat_ngram_size,
    encoder_input_ids=None,
    bad_words_ids=None,
    min_length=decoder.config.min_length,
    max_length=decoder.config.max_length,
    eos_token_id=decoder.config.eos_token_id,
    forced_bos_token_id=decoder.config.forced_bos_token_id,
    forced_eos_token_id=decoder.config.forced_eos_token_id,
    prefix_allowed_tokens_fn=None,
    num_beams=decoder.config.num_beams,
    num_beam_groups=decoder.config.num_beam_groups,
    diversity_penalty=decoder.config.diversity_penalty,
    remove_invalid_values=decoder.config.remove_invalid_values,
)(decoder_input_ids, next_token_logits)

In [207]:
torch.argmax(next_tokens_scores, dim=-1)

tensor([6743])

In [211]:
decoder_input_ids

tensor([[3]])

In [212]:
next_decoder_input_ids

tensor([6743])

In [264]:
splitted_input_ids[0].size()

torch.Size([1, 156])

## Experiment 3
Split text before tokenization

In [273]:
chunk_size = int(math.ceil(len(text) / 8))
splitted_texts = [text[0+i:chunk_size+i] for i in range(0, len(text), chunk_size)]

In [280]:
tokenized_content = tokenizer(splitted_texts, add_special_tokens=True, return_tensors="pt", padding=True, truncation=True)
tokenized_content.input_ids.size()

torch.Size([8, 166])

In [287]:
generated = autoencoder.generate(tokenized_content.input_ids, attention_mask=tokenized_content.attention_mask, max_length=166, decoder_start_token_id=3)

In [288]:
reconstructed_text = ' '.join(tokenizer.batch_decode(generated, skip_special_tokens=True))

In [316]:
encoded_vectors = encoder(tokenized_content.input_ids, attention_mask=tokenized_content.attention_mask).last_hidden_state
mean = torch.mean(encoded_vectors, dim=0, keepdim=True)

#decoder_input_ids = torch.tensor([[autoencoder.config.encoder.bos_token_id]])
# this is not working... try starting with the first words
decoder_input_ids = tokenized_content.input_ids[0][:10].reshape(1,10)

In [292]:
mean.shape

torch.Size([1, 166, 768])

In [317]:
i = 0
max_i = 166

while i < max_i:
    i = i+1
    outputs = decoder(decoder_input_ids, encoder_hidden_states=mean)
    next_token_logits = outputs.logits[:,-1:]
    next_tokens_scores = decoder._get_logits_processor(
        repetition_penalty=decoder.config.repetition_penalty,
        no_repeat_ngram_size=decoder.config.no_repeat_ngram_size,
        encoder_no_repeat_ngram_size=decoder.config.encoder_no_repeat_ngram_size,
        encoder_input_ids=None,
        bad_words_ids=None,
        min_length=decoder.config.min_length,
        max_length=decoder.config.max_length,
        eos_token_id=decoder.config.eos_token_id,
        forced_bos_token_id=decoder.config.forced_bos_token_id,
        forced_eos_token_id=decoder.config.forced_eos_token_id,
        prefix_allowed_tokens_fn=None,
        num_beams=decoder.config.num_beams,
        num_beam_groups=decoder.config.num_beam_groups,
        diversity_penalty=decoder.config.diversity_penalty,
        remove_invalid_values=decoder.config.remove_invalid_values,
    )(decoder_input_ids, next_token_logits)
    next_decoder_input_ids = torch.argmax(next_tokens_scores, axis=-1)
    decoder_input_ids = torch.cat([decoder_input_ids, next_decoder_input_ids], axis=-1)
    if next_decoder_input_ids[0][0] == decoder.config.pad_token_id:
        print("End of Summary!")
        break

In [318]:
summary = tokenizer.decode(decoder_input_ids[0], skip_special_tokens=False)
summary

'[CLS] STIKO - Mitglied Zepp [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk )sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk )sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk )sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sksk [SEP]sk )sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk )sk [SEP]sk [SEP]sk )sk [SEP]sk [SEP]sk [SEP]sk [SEP]sk [SEP]'

In [301]:
decoder_input_ids

tensor([[    3,    24, 26943, 26946]])

In [315]:
torch.argmax(decoder(tokenized_content.input_ids[0][:10].reshape(1,10), encoder_hidden_states=mean).logits[:,-1:], axis=-1)

tensor([[4]])

In [299]:
decoder_input_ids.size()

torch.Size([4])

In [312]:
tokenizer.decode(tokenized_content.input_ids[0][:10])

'[CLS] STIKO - Mitglied Zepp'